In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('neo.csv')

In [3]:
df.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True


In [4]:
df['name']

0        162635 (2000 SS164)
1          277475 (2005 WK4)
2         512244 (2015 YE18)
3                (2012 BV13)
4                (2014 GE35)
                ...         
90831             (2016 VX1)
90832             (2019 AD3)
90833             (2020 JP3)
90834             (2021 CN5)
90835             (2021 TW7)
Name: name, Length: 90836, dtype: object

In [5]:
df['year'] = df['name'].str.split('(', expand=True)[1].str.split(' ', expand=True)[0]
df['number'] = df['name'].str.split('(', expand=True)[1].str.split(' ', expand=True)[1].str.split(')',expand=True)[0]

In [6]:
df.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous,year,number
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False,2000,SS164
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True,2005,WK4
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False,2015,YE18
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False,2012,BV13
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True,2014,GE35


In [7]:
df['year'] = df.year.apply(lambda x: x if x.isnumeric() else 0)

In [8]:
df['year'] = df['year'].apply(int)

In [9]:
import tensorflow as tf
from tensorflow import keras

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [11]:
X = df.drop(columns = 'hazardous')
y = df['hazardous']

In [12]:
X.dtypes

id                      int64
name                   object
est_diameter_min      float64
est_diameter_max      float64
relative_velocity     float64
miss_distance         float64
orbiting_body          object
sentry_object            bool
absolute_magnitude    float64
year                    int64
number                 object
dtype: object

In [13]:
num_columns = X.select_dtypes(include=['int64', 'float64']).columns
cat_columns = X.select_dtypes(include=['object', 'bool']).columns

In [14]:
num_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('imputer', SimpleImputer(strategy='median'))
])
cat_pipeline = Pipeline([
    ('oh_encoder', OneHotEncoder())
])

In [15]:
X[num_columns] = num_pipeline.fit_transform(X[num_columns])

In [16]:
cat_columns

Index(['name', 'orbiting_body', 'sentry_object', 'number'], dtype='object')

In [17]:
X['name'] = cat_pipeline.fit_transform(X[['name']]).toarray()

In [18]:
X['sentry_object'] = cat_pipeline.fit_transform(X[['sentry_object']]).toarray()
X['orbiting_body'] = cat_pipeline.fit_transform(X[['orbiting_body']]).toarray()
X['number'] = cat_pipeline.fit_transform(X[['number']]).toarray()

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
train_X, test_X, train_Y, test_Y = train_test_split(X, y,\
                                                 test_size=0.2, random_state=0)

In [56]:
train_X.shape

(72668, 11)

In [96]:
from keras.layers import Dense, Activation, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.layers import BatchNormalization

In [101]:
model = keras.models.Sequential([
    keras.Input(shape=train_X.shape[1:]),
    Dense(units=100, activation='selu', use_bias=False),
    BatchNormalization(),
    Dense(units=50, activation='selu', use_bias=False),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

In [102]:
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='rmsprop')

In [103]:
model.fit(train_X, train_Y, epochs=20, validation_data=(test_X, test_Y))

Epoch 1/20
2271/2271 [==============================] - 3s 997us/step - loss: 0.2334 - accuracy: 0.8983 - val_loss: 0.2178 - val_accuracy: 0.9075
Epoch 2/20
2271/2271 [==============================] - 2s 948us/step - loss: 0.2042 - accuracy: 0.9063 - val_loss: 0.2107 - val_accuracy: 0.9104
Epoch 3/20
2271/2271 [==============================] - 2s 939us/step - loss: 0.1979 - accuracy: 0.9084 - val_loss: 0.2008 - val_accuracy: 0.9075
Epoch 4/20
2271/2271 [==============================] - 2s 954us/step - loss: 0.1942 - accuracy: 0.9089 - val_loss: 0.2045 - val_accuracy: 0.9114
Epoch 5/20
2271/2271 [==============================] - 2s 947us/step - loss: 0.1937 - accuracy: 0.9090 - val_loss: 0.2026 - val_accuracy: 0.9119
Epoch 6/20
2271/2271 [==============================] - 2s 938us/step - loss: 0.1914 - accuracy: 0.9103 - val_loss: 0.2165 - val_accuracy: 0.9119
Epoch 7/20
2271/2271 [==============================] - 2s 963us/step - loss: 0.1905 - accuracy: 0.9102 - val_loss: 0.2061 -